In [ ]:
!pip install pytrends

In [ ]:
!pip install seaborn

In [3]:
from pytrends.request import TrendReq
import pandas as pd
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import json
import time

In [4]:
timezone_offset = -330 # INDIA
# timezone_offset = -480 # Phillipines

pytrends = TrendReq(hl='en-US', tz=timezone_offset, retries=2, backoff_factor=0.5)

In [5]:
date_entry = input('Enter a start date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date1 = datetime.date(year, month, day)

Enter a start date in YYYY-MM-DD format: 2020-01-01


In [6]:
date_entry = input('Enter a end date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date2 = datetime.date(year, month, day)

Enter a end date in YYYY-MM-DD format: 2020-03-01


In [7]:
total_t_frame = date1.strftime("%Y-%m-%d") + " " + date2.strftime("%Y-%m-%d")
total_t_frame

'2020-01-01 2020-03-01'

In [8]:
enddates = pd.date_range(date1, date2 , freq='1M').tolist()
startdates = pd.to_datetime([x.date() - pd.tseries.offsets.MonthBegin(1) for x in enddates]).tolist()

In [9]:
t_frames = []
for i in range(len(startdates)):
    s = startdates[i].date().strftime("%Y-%m-%d")
    e = enddates[i].date().strftime("%Y-%m-%d")
    t_frames.append(s + " " + e)

In [10]:
t_frames

['2020-01-01 2020-01-31', '2020-02-01 2020-02-29']

In [15]:
cat_df = pd.read_csv("categories_defined.csv")
cat_names = cat_df["name"].tolist()
cat_ids = cat_df["id"].tolist()

In [12]:
def get_related_queries(catno, timeframe):
    """
    Gets related queries
    """
    kw_list = [""]
    # Create Pytrend Client
    pytrends.build_payload(cat=catno, kw_list=kw_list, timeframe=timeframe, geo="PH")
    
    # Form Request
    related_payload = dict()
    request_json = pytrends.related_queries_widget_list[0]
    related_payload["req"] = json.dumps(request_json["request"])
    related_payload["token"] = request_json["token"]
    related_payload["tz"] = pytrends.tz
    
    # Send Request
    req_json = pytrends._get_data(
        url=TrendReq.RELATED_QUERIES_URL,
        method=TrendReq.GET_METHOD,
        trim_chars=5,
        params=related_payload,
    )
    
    # return req_json
    # Tabulate Rising & Top searches.
    try:
        top_df = pd.DataFrame(req_json["default"]["rankedList"][0]["rankedKeyword"])
        top_df = top_df[["query", "formattedValue"]]
    except KeyError:
        # in case no top queries are found, the lines above will throw a KeyError
        top_df = pd.DataFrame(columns=["query", "formattedValue"])

    # rising queries
    try:
        rising_df = pd.DataFrame(req_json['default']['rankedList'][1]["rankedKeyword"])
        rising_df = rising_df[['query', 'formattedValue']]
    except KeyError:
        # in case no rising queries are found, the lines above will throw a KeyError
        rising_df = pd.DataFrame(columns=["query", "formattedValue"])

    return(top_df,rising_df)

In [13]:
datetime.datetime.strptime(t_frames[0].split()[0], "%Y-%m-%d").strftime("%b-%Y")

'Jan-2020'

In [14]:
related_queries_monthly = pd.DataFrame()
ctr = 0
for i in range(len(cat_ids)):
    print(i, cat_ids[i], cat_names[i])
    rq_top_df, rq_rising_df = get_related_queries(cat_ids[i], total_t_frame)
    
    if(len(rq_top_df)==0 or len(rq_top_df)==0):
        print("skipping ", cat_names[i], cat_ids[i])
        continue
    
    for j in range(len(t_frames)):
        ctr += 1
        if(ctr%20==0):
            time.sleep(20)
            
        rq_top_df, rq_rising_df = get_related_queries(cat_ids[i], t_frames[j])
        if(len(rq_top_df)==0):
            print(cat_names[i], "has no top queries in", t_frames[j])
        if(len(rq_rising_df)==0):
            print(cat_names[i], "has no rising queries in",  t_frames[j])

        rq_top_df.columns = ["value", "subject"]
        rq_rising_df.columns = ["value", "subject"]

        rq_top_df["related_queries"] = "top"
        rq_rising_df["related_queries"] = "rising"

        rq_df = rq_top_df.append(rq_rising_df)

        rq_df["geo"] = "PH"
        rq_df["keyword"] = cat_names[i]
        rq_df["category"] = cat_ids[i]
        rq_df["timeframe"] = datetime.datetime.strptime(t_frames[j].split()[0], "%Y-%m-%d").strftime("%b-%Y")
        related_queries_monthly = related_queries_monthly.append(rq_df)

0 277 Alcoholic Beverages


ReadTimeout: HTTPSConnectionPool(host='trends.google.com', port=443): Read timed out. (read timeout=2)

In [35]:
related_queries_monthly.reset_index().to_csv("related_queries_monthly.csv")